In [1]:
import operator
import csv

input1 = []
input2 = []

inputFile  = "test-input-1.csv"

outputFile = "output.csv"


with open(inputFile, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='\t')
    for row in spamreader:
        input1.append(list(filter(None, row[0].split(','))))
        
input2 = input1.pop(-1)

c_s_p = {}

for i in range(len(input1)):
    c_s_p[input1[i][0]] = input1[i][1::]

numOfRooms = len(input2)

def SubjectsWithLeastSlots(assignment, csp):
    unassignedSubjects = []
    for k in assignment.keys():
        if assignment[k]=="null":
            unassignedSubjects.append(k[0])

    subjectsWithLeastSlots = []
    MinNumOfSlots = float('inf')

    for key in unassignedSubjects:
        L = len(csp[key])
        if L < MinNumOfSlots:
            MinNumOfSlots = L

    for key in unassignedSubjects:
        if len(csp[key]) == MinNumOfSlots:
            subjectsWithLeastSlots.append(key)
            
    return subjectsWithLeastSlots

def CalConstraintVal(key, dic):
    constraintVal = 0
    numOfOptionals = 0
    C_or_O = dic[key][0]
    for i in range(1, len(dic[key])):
        for k in dic.keys():
            if k!=key:
                for z in range(1, len(dic[k])):
                    if dic[key][i] == dic[k][z]:
                        if C_or_O == "o" and dic[k][0] == "o":
                            constraintVal += 0.5
                            numOfOptionals += 1
                        else:
                            constraintVal += 1
                            
    return (constraintVal, numOfOptionals)  #take max optionalVal to tie break


def slotConflictValue(subject, timeSlot, csp):
    constraintVal = 0
    numOfOptionals = 0
    for k in csp.keys():
        for z in range(1, len(csp[k])):
            if timeSlot == csp[k][z]:
                if csp[subject][0] == "o" and csp[k][0] == "o":
                    constraintVal += 0.5
                    numOfOptionals += 1
                else:
                    constraintVal += 1               
    return (constraintVal, numOfOptionals)  #take min optionalVal to tie break  

def orderedTimeSlots(subject, assignment, csp):
    timeSlotsForSubject = csp[subject][1::]      
    slot_constraint_assignment = {}
    for slot in timeSlotsForSubject:
        slot_constraint_assignment[slot] = slotConflictValue(subject, slot, csp)
    #print (slot_constraint_assignment)
    ordered_list = sorted(slot_constraint_assignment.items(), key=operator.itemgetter(1))  #[('Tu3', (3, 2)), ('Th4', (3, 5))]
    return [ordered_list[i][0] for i in range(len(ordered_list))]

def SelectUnassignedSubject(assignment, csp):
    keys = SubjectsWithLeastSlots(assignment, csp)   #array of keys with same number of remaining slots
    tup = (-float('inf'),float('inf'))
    MostConstrainedSubject = ""             #ONLY ONE KEY WITH HIGHSET CONSTRANIT CAN BE FOUND YET
    for k in keys:
        if CalConstraintVal(k, csp)[0] > tup[0]:
            tup = CalConstraintVal(k, csp)
            MostConstrainedSubject = k
        elif((CalConstraintVal(k, csp)[0] == tup[0])) and (CalConstraintVal(k, csp)[1] < tup[1]):
            tup = CalConstraintVal(k, csp)
            MostConstrainedSubject = k
    return MostConstrainedSubject


def isComplete(assignment):
    if "null" in assignment.values():
        return False
    else:
        return True

def isConsistent(slot, assignment):
    if slot not in assignment.values():
        return True
    elif slot in assignment.values():
        count = 0
        for sub, tSlot in assignment.items():    # for name, age in list.items():  (for Python 3.x)
            if (tSlot == slot) and (sub[1] == "o"):
                count += 1
        if (count > 0) and (count < numOfRooms):
            return True
        else:
            return False                         #either count = 0 or  count = numOfRooms

def updateDomains(csp, subject, slot):
    if subject[1] == "c":
        for k in csp.keys():
            if slot in csp[k]:
                csp[k].remove(slot)
    return csp

def printAssignment(assignment):
    slot_duplicate = {}
    finalOutput = []
    for sub, tSlot in assignment.items():
        if tSlot not in slot_duplicate:
            slot_duplicate[tSlot] = 1
        else:
            slot_duplicate[tSlot] += 1
    with open(outputFile, 'w') as csvfile:
        fieldnames = ['SUBJECT', 'TIME_SLOT', 'ROOM']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for sub in assignment.keys():
            finalOutput.append([sub[0], assignment[sub], "R"+str(slot_duplicate[assignment[sub]])])
            writer.writerow({'SUBJECT': sub[0], 'TIME_SLOT': assignment[sub], 'ROOM': "R"+str(slot_duplicate[assignment[sub]])})
            slot_duplicate[assignment[sub]] -= 1
    #print(finalOutput)
    return finalOutput
                       
def RecursiveBacktracking(assignment, csp):
    if isComplete(assignment):
        return True
    else:
        subject = SelectUnassignedSubject(assignment, csp)
        for slot in orderedTimeSlots(subject, assignment, csp):
            #print (slot)
            if isConsistent(slot, assignment):
                assignment[(subject, csp[subject][0])] = slot
                temp = csp
                csp = updateDomains(csp, (subject, csp[subject][0]), slot)
                if RecursiveBacktracking(assignment, csp):
                    return True
                else:
                    assignment[(subject, csp[subject][0])] = "null"                
            #check whether consistent
            #if yes, (var = val), (remove domains in csp) recursive backtracking
            #if no, remove (var = val) and replace csp with original
    return False 
    
def backTrackingSearch(csp):
    assignment = {}
    for subject in csp:
        assignment[(subject, csp[subject][0])] = "null"
    #print (assignment)
    if RecursiveBacktracking(assignment, csp) == True:
        print ("some result found")
        printAssignment(assignment) 
    else:
        print ("no result found")
        with open('output.csv', 'w') as csvfile:
            fieldnames = ['no result found']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

backTrackingSearch(c_s_p)

print("DONE!, See given output file for results!")
input("Press any key to exit!")




some result found
DONE!, See given output file for results!
Press any key to exit!5


'5'

In [2]:
aa = [('Tu3', (3, 2)), ('Th4', (3, 5)),('Th4', (2, 5)),('Th4', (3, 1))]
sorted(aa, key=operator.itemgetter(1))

[('Th4', (2, 5)), ('Th4', (3, 1)), ('Tu3', (3, 2)), ('Th4', (3, 5))]

In [ ]:
def to_int(val): #Accepts type errors and value errors so thats the program continues.
    try:
        return float(val)
    except(TypeError, ValueError):
        return -1

def read_route_data(file_name):  # Reads in routes.txt file and checks for formating errors
    routes = []                  # Initialise routes list
    with open(file_name) as a:
        lines = a.readlines()
        for line in lines: # Checking line formate
            line = line[:-1]
            l = line.split(',') # Checking line format with comma to define keys
            if len(l) != 2:     # If each line is not split exactly two times it will cause an error.
                print("Error reading data."
                      "\nPlease ensure each line of routes.txt contains a route number, followed by a comma, followed by a happy ratio"
                      "\nand that no route is repeated throughout the file")
                return -1
            if to_int(l[0]) == -1 or to_int(l[1]) == -1 :  # If either key contains type error it will cause an error.
                print("Error reading data."
                      "\nPlease ensure each line of routes.txt contains a route number, followed by a comma, followed by a happy ratio"
                      "\nand that no route is repeated throughout the file")
                return -1
            if (len(routes)!= len(set(routes))):
                print("Error reading data."
                      "\nPlease ensure each line of routes.txt contains a route number, followed by a comma, followed by a happy ratio"
                      "\nand that no route is repeated throughout the file")
                return -1
            routes.append({'route_number': int(l[0]), 'happy_ratio': float(l[1])}) # Append route_number and happy_ratio to associated key in dictionary
    return routes


def sort_route_data(routes):
    return sorted(routes, key=lambda item: item['happy_ratio']) # Returns list sorted using the algorithm sorted
   

if __name__ == '__main__':
    routes = read_route_data('routes.txt')
    if routes != -1: # If formatting is correct
        updated = sort_route_data(routes) # run code in sort_route_data
        while True:
            n = input("How many routes can have an extra bus? ")
            if to_int(n) == -1: # Input is checked for typeError and valueError
                print("Invalid value. Please enter a non-negative integer")
                continue
            n = int(n)
            if n < 0 or n >= len(routes):   # Input is checked for more then one less then number of routes listed
                print("Invalid value. Please enter a non-negative integer")
                continue
            break
        new = []  # Initialise new list for route number output
        index = 0
        print("You should add busses for the following routes: ")
        while index < len(updated) and n > 0:
            if updated[index]['happy_ratio'] == 0:
                new.append(updated[index])
            else:
                print(updated[index]['route_number'])
                n -= 1
            index += 1
        index = 0
        while n:
            print(new[-1 - index]['route_number'])
            n -= 1
            index += 1
